# Segmenting and Clustering Neighborhoods in London

In [255]:
# import the necessary libraries
# !pip install watson-developer-cloud==1.5
# commenting off since it's already installed

In [256]:
# web scrapping
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [257]:
# Looks like not all websites are scrappable
# page = requests.get('https://en.wikipedia.org/wiki/List_of_areas_of_London')
# Create a BeautifulSoup object
# soup = BeautifulSoup(page.text, 'html.parser')
# print(soup.prettify())

In [258]:
# The code was removed by Watson Studio for sharing.

,id,name,county,country,grid_reference,easting,northing,latitude,longitude,elevation,postcode_sector,local_government_area,nuts_region,type
0,1,Aaron's Hill,Surrey,England,SU957435,495783,143522,51.18291,-0.63098,78,GU7 2,Waverley District,South East,Suburban Area
1,2,Abbas Combe,Somerset,England,ST707226,370749,122688,51.00283,-2.41825,91,BA8 0,South Somerset District,South West,Village
2,3,Abberley,Worcestershire,England,SO744675,374477,267522,52.30522,-2.37574,152,WR6 6,Malvern Hills District,West Midlands,Village
3,4,Abberton,Essex,England,TM006190,600637,219093,51.83440,0.91066,44,CO5 7,Colchester District,Eastern,Village
4,5,Abberton,Worcestershire,England,SO995534,399538,253477,52.17955,-2.00817,68,WR10 2,Wychavon District,West Midlands,Hamlet


In [259]:

df_data_1.head()
#df = df_data_1[df_data_1["county"]=="Greater London"]
df = df_data_1[df_data_1["country"]=="England"]
df.shape

(1196, 14)

In [260]:
df.drop_duplicates(subset='name', keep='first', inplace=True)
df = df.head(100)
df.shape

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


(100, 14)

In [261]:
# plot on folium the neighborhoods of London, importing the libraries
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
# commenting off since it's already completed
import folium # map rendering library

In [262]:
latitude = 51.507350
longitude = -0.127756

# create map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['latitude'], df['longitude'], df['country'], df['name']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

In [263]:
# The code was removed by Watson Studio for sharing.

In [264]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng,
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [265]:
london_venues = getNearbyVenues(names=df['name'],latitudes=df['latitude'],longitudes=df['longitude'])

london_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Abbas Combe,51.00283,-2.41825,Templecombe Railway Station (TMC),51.001564,-2.417480,Train Station
1,Abbas Combe,51.00283,-2.41825,The Royal Wessex,51.001735,-2.415663,Pub
2,Abbas Combe,51.00283,-2.41825,The Templars Retreat,51.001755,-2.415475,Pub
3,Abberley,52.30522,-2.37574,Abberley,52.304917,-2.374229,Scenic Lookout
4,Abberton,51.83440,0.91066,Lucy's Health & Beauty,51.835525,0.916322,Spa


In [266]:
london_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Ab Kettleby,2,2,2,2,2,2
Abbas Combe,3,3,3,3,3,3
Abberley,1,1,1,1,1,1
Abberton,1,1,1,1,1,1
Abbey End,23,23,23,23,23,23
Abbey Field,9,9,9,9,9,9
Abbey Fields,14,14,14,14,14,14
Abbey Gate,1,1,1,1,1,1
Abbey Hey,2,2,2,2,2,2


In [267]:
print('There are {} uniques categories.'.format(len(london_venues['Venue Category'].unique())))

There are 94 uniques categories.


# Analyze each neighborhood

In [268]:
# one hot encoding
london_onehot = pd.get_dummies(london_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
london_onehot['Neighbourhood'] = london_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [london_onehot.columns[-1]] + list(london_onehot.columns[:-1])
london_onehot = london_onehot[fixed_columns]

london_onehot.head()

,Neighbourhood,Athletics & Sports,Auto Garage,Bakery,Bar,Bed & Breakfast,Bookstore,Bowling Alley,Breakfast Spot,Brewery,...,Thai Restaurant,Theme Park,Trail,Train Station,Tram Station,Transportation Service,Vietnamese Restaurant,Warehouse Store,Wine Shop,Yoga Studio
0,Abbas Combe,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,Abbas Combe,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Abbas Combe,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Abberley,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Abberton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [269]:
london_onehot.shape

(302, 95)

In [270]:
london_grouped = london_onehot.groupby('Neighbourhood').mean().reset_index()
london_grouped.head()

,Neighbourhood,Athletics & Sports,Auto Garage,Bakery,Bar,Bed & Breakfast,Bookstore,Bowling Alley,Breakfast Spot,Brewery,...,Thai Restaurant,Theme Park,Trail,Train Station,Tram Station,Transportation Service,Vietnamese Restaurant,Warehouse Store,Wine Shop,Yoga Studio
0,Ab Kettleby,0.0,0.5,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,Abbas Combe,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0
2,Abberley,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,Abberton,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,Abbey End,0.0,0.0,0.0,0.0,0.0,0.043478,0.0,0.0,0.0,...,0.043478,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [271]:
london_grouped.shape

(70, 95)

In [272]:
num_top_venues = 10

for hood in london_grouped['Neighbourhood']:
    #print("----"+hood+"----")
    temp = london_grouped[london_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    #print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    #print('\n')

In [273]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [274]:
import numpy as np # library to handle data in a vectorized manner

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = london_grouped['Neighbourhood']

for ind in np.arange(london_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ab Kettleby,Auto Garage,Pub,Yoga Studio,French Restaurant,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant
1,Abbas Combe,Pub,Train Station,French Restaurant,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
2,Abberley,Scenic Lookout,Yoga Studio,Cricket Ground,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
3,Abberton,Spa,Yoga Studio,Cricket Ground,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
4,Abbey End,Indian Restaurant,Hotel,Pub,Coffee Shop,Supermarket,Furniture / Home Store,Park,Sandwich Place,Cocktail Bar,Shopping Mall


In [275]:
neighbourhoods_venues_sorted.shape

(70, 11)

# Cluster neighborhoods

In [276]:
london_grouped.head()

,Neighbourhood,Athletics & Sports,Auto Garage,Bakery,Bar,Bed & Breakfast,Bookstore,Bowling Alley,Breakfast Spot,Brewery,...,Thai Restaurant,Theme Park,Trail,Train Station,Tram Station,Transportation Service,Vietnamese Restaurant,Warehouse Store,Wine Shop,Yoga Studio
0,Ab Kettleby,0.0,0.5,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,Abbas Combe,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0
2,Abberley,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,Abberton,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,Abbey End,0.0,0.0,0.0,0.0,0.0,0.043478,0.0,0.0,0.0,...,0.043478,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [277]:
london_grouped[['Food']]

,Food
0,0.000000
1,0.000000
2,0.000000
3,0.000000
4,0.000000
5,0.111111
6,0.000000
7,0.000000
8,0.000000
9,0.000000


In [278]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [279]:
# set number of clusters
kclusters = 5

london_grouped_clustering = london_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

london_merged = df



In [280]:
london_merged.head()

,id,name,county,country,grid_reference,easting,northing,latitude,longitude,elevation,postcode_sector,local_government_area,nuts_region,type
0,1,Aaron's Hill,Surrey,England,SU957435,495783,143522,51.18291,-0.63098,78,GU7 2,Waverley District,South East,Suburban Area
1,2,Abbas Combe,Somerset,England,ST707226,370749,122688,51.00283,-2.41825,91,BA8 0,South Somerset District,South West,Village
2,3,Abberley,Worcestershire,England,SO744675,374477,267522,52.30522,-2.37574,152,WR6 6,Malvern Hills District,West Midlands,Village
3,4,Abberton,Essex,England,TM006190,600637,219093,51.83440,0.91066,44,CO5 7,Colchester District,Eastern,Village
5,6,Abberwick,Northumberland,England,NU129132,412938,613277,55.41325,-1.79720,93,NE66 2,Northumberland,North East,Locality


In [281]:

neighbourhoods_venues_sorted.head()


,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,3,Ab Kettleby,Auto Garage,Pub,Yoga Studio,French Restaurant,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant
1,3,Abbas Combe,Pub,Train Station,French Restaurant,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
2,1,Abberley,Scenic Lookout,Yoga Studio,Cricket Ground,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
3,0,Abberton,Spa,Yoga Studio,Cricket Ground,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
4,0,Abbey End,Indian Restaurant,Hotel,Pub,Coffee Shop,Supermarket,Furniture / Home Store,Park,Sandwich Place,Cocktail Bar,Shopping Mall


In [282]:

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
london_merged = london_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='name')
london_merged.dropna(inplace=True)
london_merged.shape # check the last columns!
# london_merged.head(50)
london_merged[['name','latitude','longitude','Cluster Labels']]

,name,latitude,longitude,Cluster Labels
1,Abbas Combe,51.00283,-2.41825,3.0
2,Abberley,52.30522,-2.37574,1.0
3,Abberton,51.83440,0.91066,0.0
10,Abbeydale,51.84615,-2.19922,0.0
13,Abbeydale Park,53.32291,-1.52539,3.0
15,Abbey End,52.34412,-1.58043,0.0
16,Abbey Field,51.88336,0.89730,3.0
17,Abbey Fields,52.34490,-1.58497,0.0
18,Abbey Gate,50.77278,-3.00118,0.0
21,Abbey Hey,53.46514,-2.15963,0.0


In [283]:
london_merged['Cluster Labels'] = london_merged['Cluster Labels'].astype(int)

In [284]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_merged['latitude'], london_merged['longitude'], london_merged['name'], london_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [285]:
london_merged.loc[london_merged['Cluster Labels'] == 0, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

,name,easting,northing,latitude,longitude,elevation,postcode_sector,local_government_area,nuts_region,type,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Abberton,600637,219093,51.83440,0.91066,44,CO5 7,Colchester District,Eastern,Village,...,Spa,Yoga Studio,Cricket Ground,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
10,Abbeydale,386373,216413,51.84615,-2.19922,47,GL4 4,Gloucester District,South West,Suburban Area,...,Playground,Indian Restaurant,Grocery Store,Pub,Bar,Bed & Breakfast,Discount Store,Dive Bar,English Restaurant,Farmers Market
15,Abbey End,428680,271865,52.34412,-1.58043,92,CV8 1,Warwick District,West Midlands,Suburban Area,...,Indian Restaurant,Hotel,Pub,Coffee Shop,Supermarket,Furniture / Home Store,Park,Sandwich Place,Cocktail Bar,Shopping Mall
17,Abbey Fields,428370,271950,52.34490,-1.58497,81,CV8 1,Warwick District,West Midlands,Suburban Area,...,Italian Restaurant,Indian Restaurant,Cocktail Bar,Park,Castle,Furniture / Home Store,Sandwich Place,Gastropub,Mediterranean Restaurant,Grocery Store
18,Abbey Gate,329500,97500,50.77278,-3.00118,55,EX13 5,East Devon District,South West,Locality,...,Home Service,Yoga Studio,Furniture / Home Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market
21,Abbey Hey,389500,396500,53.46514,-2.15963,87,M18 8,Manchester District,North West,Locality,...,Performing Arts Venue,Stables,Yoga Studio,Deli / Bodega,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
23,Abbey Hulton,390708,348651,53.03506,-2.14002,136,ST2 8,City of Stoke-on-Trent,West Midlands,Suburban Area,...,Café,Gym / Fitness Center,Concert Hall,Grocery Store,Furniture / Home Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
24,Abbey Mead,504500,167500,51.39692,-0.49929,14,KT16 8,Runnymede District,South East,Locality,...,Bed & Breakfast,Yoga Studio,Furniture / Home Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market
25,Abbey Meads,414197,189131,51.60084,-1.79642,107,SN25 4,Swindon,South West,Suburban Area,...,Pub,Restaurant,Grocery Store,English Restaurant,Chinese Restaurant,Food,Department Store,Discount Store,Dive Bar,Farmers Market
28,Abbeystead,356356,454382,53.98364,-2.66703,115,LA2 9,Lancaster District,North West,Hamlet,...,Trail,Yoga Studio,French Restaurant,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop


In [286]:
london_merged.loc[london_merged['Cluster Labels'] == 1, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

,name,easting,northing,latitude,longitude,elevation,postcode_sector,local_government_area,nuts_region,type,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Abberley,374477,267522,52.30522,-2.37574,152,WR6 6,Malvern Hills District,West Midlands,Village,...,Scenic Lookout,Yoga Studio,Cricket Ground,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
43,Abbots Leigh,353799,173737,51.46071,-2.66640,86,BS8 3,North Somerset,South West,Village,...,Scenic Lookout,Bus Stop,Yoga Studio,French Restaurant,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop


In [287]:
london_merged.loc[london_merged['Cluster Labels'] == 2, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

,name,easting,northing,latitude,longitude,elevation,postcode_sector,local_government_area,nuts_region,type,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
49,Abbots Ripton,523054,278110,52.38720,-0.19329,42,PE28 2,Huntingdonshire District,Eastern,Village,...,Restaurant,Yoga Studio,Cricket Ground,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
289,Acresford,429933,313139,52.71509,-1.55833,79,DE12 8,North West Leicestershire District,East Midlands,Village,...,Restaurant,Yoga Studio,Cricket Ground,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
303,Acton Bridge,359650,375345,53.27354,-2.60652,46,CW8 3,Cheshire West and Chester,North West,Village,...,Restaurant,Yoga Studio,Cricket Ground,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop


In [288]:
london_merged.loc[london_merged['Cluster Labels'] == 3, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

,name,easting,northing,latitude,longitude,elevation,postcode_sector,local_government_area,nuts_region,type,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Abbas Combe,370749,122688,51.00283,-2.41825,91,BA8 0,South Somerset District,South West,Village,...,Pub,Train Station,French Restaurant,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
13,Abbeydale Park,431711,380770,53.32291,-1.52539,143,S17 3,Sheffield District,Yorkshire and the Humber,Suburban Area,...,Cricket Ground,Grocery Store,Pub,Food,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant
16,Abbey Field,599500,224500,51.88336,0.89730,38,CO2 7,Colchester District,Eastern,Locality,...,Pub,Bar,Furniture / Home Store,Italian Restaurant,Food,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant
29,Abbeytown,317279,550746,54.84466,-3.28970,14,CA7 4,Allerdale District,North West,Village,...,Pub,Convenience Store,Yoga Studio,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
35,Abbots Bromley,408370,324413,52.81719,-1.87725,119,WS15 3,East Staffordshire District,West Midlands,Village,...,Pub,Bed & Breakfast,Brewery,French Restaurant,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant
38,Abbotsford,530382,120922,50.97296,-0.14434,29,RH15 8,Mid Sussex District,South East,Hamlet,...,Golf Course,Pub,Yoga Studio,Food,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant
39,Abbotsham,242457,126488,51.01613,-4.24738,61,EX39 5,Torridge District,South West,Village,...,Pub,Theme Park,Hotel,Bakery,Deli / Bodega,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant
42,Abbots Langley,509444,201966,51.70577,-0.41754,128,WD5 0,Three Rivers District,Eastern,Village,...,Pub,Chinese Restaurant,Convenience Store,Indian Restaurant,Café,Health & Beauty Service,Discount Store,Dive Bar,Home Service,English Restaurant
44,Abbotsley,522933,256521,52.19323,-0.20294,45,PE19 6,Huntingdonshire District,Eastern,Village,...,Bus Stop,Pub,Yoga Studio,French Restaurant,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant
46,Abbot's Mead,339886,368119,53.20672,-2.90149,8,CH1 4,Cheshire West and Chester,North West,Suburban Area,...,Pub,Nature Preserve,Rental Car Location,Gym / Fitness Center,Flea Market,Deli / Bodega,Department Store,Discount Store,Dive Bar,English Restaurant


In [289]:
london_merged_cluster_5 = london_merged.loc[london_merged['Cluster Labels'] == 4, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]
london_merged_cluster_5

,name,easting,northing,latitude,longitude,elevation,postcode_sector,local_government_area,nuts_region,type,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,Abbotskerswell,285694,68785,50.50767,-3.61335,31,TQ12 5,Teignbridge District,South West,Village,...,Pub,Yoga Studio,French Restaurant,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
55,Abbots Worthy,449752,132684,51.09132,-1.29097,53,SO21 1,Winchester District,South East,Village,...,Pub,Yoga Studio,French Restaurant,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
107,Aberford,443343,437258,53.82978,-1.34296,43,LS25 3,Leeds District,Yorkshire and the Humber,Village,...,Pub,Yoga Studio,French Restaurant,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
164,Abington Pigotts,530720,244481,52.08326,-0.09371,28,SG8 0,South Cambridgeshire District,Eastern,Village,...,Pub,Yoga Studio,French Restaurant,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
182,Abthorpe,464964,246445,52.11252,-1.05275,121,NN12 8,South Northamptonshire District,East Midlands,Village,...,Pub,Yoga Studio,French Restaurant,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
185,Aby,541214,378361,53.28342,0.11672,15,LN13 0,East Lindsey District,East Midlands,Village,...,Pub,Yoga Studio,French Restaurant,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
188,Acaster Malbis,458778,445447,53.90186,-1.10691,10,YO23 2,York,Yorkshire and the Humber,Village,...,Pub,Yoga Studio,French Restaurant,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
274,Ackleton,377045,298665,52.58530,-2.34023,91,WV6 7,Shropshire,West Midlands,Village,...,Pub,Yoga Studio,French Restaurant,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
292,Acton,363235,353000,53.07296,-2.55019,63,CW5 8,Cheshire East,North West,Village,...,Pub,Yoga Studio,French Restaurant,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop


In [290]:
# Find the neighborhood that is the most accessible, sort by transport column and plot

In [291]:
london_grouped_busstoped = london_grouped.sort_values(by='Bus Stop',ascending=False)
london_grouped_busstoped[['Neighbourhood','Bus Stop']].head()

,Neighbourhood,Bus Stop
23,Abbots Leigh,0.500000
31,Abbotsley,0.500000
42,Abington Vale,0.200000
58,Acle,0.200000
69,Acton Green,0.029412


In [292]:
london_grouped_busstoped = london_grouped.sort_values(by='Train Station',ascending=False)
london_grouped_busstoped[['Neighbourhood','Train Station']].head()

,Neighbourhood,Train Station
55,Acklington,0.500000
1,Abbas Combe,0.333333
32,Abbotsmead,0.333333
52,Accrington,0.083333
69,Acton Green,0.029412


In [293]:
# Find the neighborhood with the most and least cafes?


In [294]:
london_merged_cluster_5[['name','latitude','longitude',
                         '1st Most Common Venue',
                         '2nd Most Common Venue',
                         '3rd Most Common Venue',
                         '4th Most Common Venue',
                         '5th Most Common Venue',
                         '6th Most Common Venue',
                         '7th Most Common Venue',
                         '8th Most Common Venue',
                         '9th Most Common Venue',
                         '10th Most Common Venue']]



,name,latitude,longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,Abbotskerswell,50.50767,-3.61335,Pub,Yoga Studio,French Restaurant,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
55,Abbots Worthy,51.09132,-1.29097,Pub,Yoga Studio,French Restaurant,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
107,Aberford,53.82978,-1.34296,Pub,Yoga Studio,French Restaurant,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
164,Abington Pigotts,52.08326,-0.09371,Pub,Yoga Studio,French Restaurant,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
182,Abthorpe,52.11252,-1.05275,Pub,Yoga Studio,French Restaurant,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
185,Aby,53.28342,0.11672,Pub,Yoga Studio,French Restaurant,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
188,Acaster Malbis,53.90186,-1.10691,Pub,Yoga Studio,French Restaurant,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
274,Ackleton,52.58530,-2.34023,Pub,Yoga Studio,French Restaurant,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
292,Acton,53.07296,-2.55019,Pub,Yoga Studio,French Restaurant,Department Store,Discount Store,Dive Bar,English Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
